In [7]:
import os
import xmltodict
import jsonlines
import pandas as pd
import jsonlines

In [6]:
!pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------------------------- ---- 9.7/11.0 MB 46.5 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 40.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------- ----------- 9.2/12.8 MB 47.7 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 40.1 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- --------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.17.0 requires protobuf>=3.20.2, which is not installed.


In [ ]:
folders = os.listdir('.')
folders = ['./'+item for item in folders if item != 'readme.txt']
data = []
for folder in folders:
    files = os.listdir(folder)
    files = [folder+'/'+item for item in files]
    for file in files:
        with open(file) as xml_file:
            data_dict = xmltodict.parse(xml_file.read())
            if 'Document' in data_dict.keys():
                if data_dict['Document']['QAPairs']:
                    if 'QAPair' in data_dict['Document']['QAPairs'].keys():
                        qas = data_dict['Document']['QAPairs']['QAPair']
                        if not isinstance(qas, list):
                            qas = [qas]
                        for qa in qas:
                            # print(qa)
                            # qa = qa['QAPair']
                            # print(qa['Question']['@qid'])
                            
                                data.append({'id': qa['Question']['@qid'],
                                        'url': data_dict['Document']['@url'],
                                        'subject': '',
                                        'topic': data_dict['Document']['Focus'],
                                        'context': '',
                                        'question': qa['Question']['#text'],
                                        'answer': qa['Answer']})

with jsonlines.open('train_data.jsonl', mode='w') as writer:
    for item in data:
        writer.write(item)
print(len(data))

In [ ]:
data = pd.read_csv('./All-2479-Answers-retrieved-from-MedQuAD.csv')
with jsonlines.open('test_data.jsonl', mode='w') as writer:
    for item in data.iterrows():
        id = item[0]
        for line in item[1]['Answer'].split('\n'):
            if 'Question:' in line:
                question = line.split('Question:')[-1].strip()
            if 'URL:' in line:
                url = line.split('URL:')[-1].strip()
            if 'Answer:' in line:
                answer = line.split('Answer:')[-1].strip()
        writer.write({'id': id,
                      'url': url,
                      'subject': '',
                      'topic': '',
                      'context': '',
                      'question': question,
                      'answer': [answer]})